# Welcome to the Lab 🥼🧪

## Market Analysis 

Let's do a deep dive on one specific market, your choice. 

Let's get into the lab and take a peak at whats going on. 

**Note** This notebook will work with any of the 70k+ markets supported by the Parcl Labs API.

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along. 

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/market_analysis.ipynb)

## Table of Contents

### [Market Overview](#market)
- Housing Stock/Mix Analysis
- Percent of Overall Market Owned by Investors
- Market Overview: Percent of all units that are rentals
- Who owns the homes? Size of investors in the market

### [Listing Overview](#listings)
- New listings analysis
- Of listings, what share do investors account for?

### [Prices Overview](#prices)
- Rental Prices
- New Listing for Sale Prices
- Purchase Prices

We will be creating charts like this: 

![Chart1](assets/purchase_price_vs_list_price.png)
![Chart2](assets/rental_listing_price.png)
![Chart3](assets/single_family_homes_percentage.png)

In [ ]:
# Environment setup
import os
import sys
import subprocess
from datetime import datetime

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')

In [ ]:
import parcllabs
import pandas as pd
import plotly.express as px
from parcllabs import ParclLabsClient

print(f"Parcl Labs Version: {parcllabs.__version__}")

In [ ]:
client = ParclLabsClient(api_key=api_key)

In [ ]:
# we are looking at homes in San Diego, so we look for the name and the type of market
results = client.search_markets.retrieve(
    query='San Diego',
    location_type='CITY',
    as_dataframe=True
)

results.head()

In [ ]:
# we see that the the first result is the city of San Diego, so we will use the parcl id of 5374167
analysis_parcl_id = results.iloc[0]['parcl_id']
name = results.iloc[0]['name']

# set default save directory
save_dir = './graphics'
new_dir_name = f"{analysis_parcl_id}-{name}-{datetime.today().date()}"
output_dir = os.path.join(save_dir, new_dir_name)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print(f"In this analysis, we are studying: {name} which has parcl id: {analysis_parcl_id}. Saving graphics to: {save_dir}")

<a id='market'></a>
#### Market Overview

Let's review housing stock mix, percent of housing stock owned by investors, and percent of all units that are rentals.

In [ ]:
# we call the API specifying the parcl id and the necessary parameters
housing_stock = client.market_metrics_housing_stock.retrieve(
    parcl_id=analysis_parcl_id,
    params={
        'limit': 100
    },
    as_dataframe=True # set to true to default to a pandas dataframe for easier manipulation
)

housing_stock.head()

In [ ]:
# calculate the percentage of each property type
housing_stock['pct_condo'] = housing_stock['condo']/housing_stock['all_properties']
housing_stock['pct_townhouse'] = housing_stock['townhouse']/housing_stock['all_properties']
housing_stock['pct_single_family'] = housing_stock['single_family']/housing_stock['all_properties']
housing_stock['pct_other'] = housing_stock['other']/housing_stock['all_properties']
housing_stock.head()


Let's analyze how the percent of single family homes as a proportion of the overall housing stock has changed
since 2019 in San Diego. The API returns monthly data for this endpoint since January 2019. We will see how the supply of housing has changed over the last 5 years. 

In [ ]:
# set some constants for the analysis, such as the logo for the chart, we have a white and blue version
labs_logo_lookup = {
    'blue': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api.png',
    'white': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api-logo-white.png'
}

# set charting constants
labs_logo_dict = dict(
        source=labs_logo_lookup['white'],
        xref="paper",
        yref="paper",
        x=0.5,  # Centering the logo below the title
        y=1.02,  # Adjust this value to position the logo just below the title
        sizex=0.15, 
        sizey=0.15,
        xanchor="center",
        yanchor="bottom"
)

# define image dimentions
media_img_size_lookup = {
    'X': {
        'width': 1600,
        'height': 900
    }
}

# optimize the visual for the platform in this case X
PLATFORM = 'X'

# set image sizes
IMG_WIDTH = media_img_size_lookup[PLATFORM]['width']
IMG_HEIGHT = media_img_size_lookup[PLATFORM]['height']

# plotting title settings
PLOT_TITLE_SETTINGS = {
        'y':0.97,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    }

In [ ]:
# Creating the line chart
fig = px.line(
    housing_stock, 
    x='date', 
    y='pct_single_family', 
    title=f'Percent of all homes that are Single Family Homes in {name}',
    labels={'pct_single_family': 'Percent of Single Family Homes'}
)

# Showing the plot
# Add the logo as a layout_image
fig.add_layout_image(
    labs_logo_dict
)

fig.update_layout(
    margin=dict(b=100),
    width=IMG_WIDTH,
    height=IMG_HEIGHT,
    legend=dict(
        x=0.01,
        y=0.99,
        traceorder="normal",
        xanchor='left',
        yanchor='top',
        title='Rolling Window (Days)'
    ),
    title=PLOT_TITLE_SETTINGS,
    plot_bgcolor='#080D16',
    paper_bgcolor='#080D16',
    font=dict(color='#FFFFFF'),
    xaxis_title='date'
)

fig.update_traces(
    line=dict(color='#FFFFFF', width=3),
    mode='lines+markers',
    marker=dict(color='#448CF2', size=5)
)


# Adding gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(tickformat=".2%")

# save fig
fig.write_image(os.path.join(output_dir, 'percent_single_family_homes.png'), width=IMG_WIDTH, height=IMG_HEIGHT)
# Show the plot
fig.show()

In [ ]:
# Now lets find out how much of the housing stock is owned by investors in San Diego
investor_ownership = client.investor_metrics_housing_stock_ownership.retrieve(
    parcl_id=analysis_parcl_id,
    params={
        'limit': 200,
    },
    as_dataframe=True
)

# the results returned by the API are in percentage points, howver for graph formatting we divide pct_ownership 
# so plotly can handles this internally
investor_ownership['pct_ownership'] = investor_ownership['pct_ownership']/100

In [ ]:
# Creating the line chart
fig = px.line(
    investor_ownership, 
    x='date', 
    y='pct_ownership', 
    title=f'Percent of all homes owned by investors in {name}',
    labels={'pct_ownership': 'Percent of all Homes Owned by Investors'}
)

# Showing the plot
# Add the logo as a layout_image
fig.add_layout_image(
    labs_logo_dict
)

fig.update_layout(
    margin=dict(b=100),
    height=IMG_HEIGHT,
    width=IMG_WIDTH,
    legend=dict(
        x=0.01,
        y=0.99,
        traceorder="normal",
        xanchor='left',
        yanchor='top',
        title='Rolling Window (Days)'
    ),
    title=PLOT_TITLE_SETTINGS,
    plot_bgcolor='#080D16',
    paper_bgcolor='#080D16',
    font=dict(color='#FFFFFF'),
    xaxis_title='date'
)

fig.update_traces(
    line=dict(color='#FFFFFF', width=3),
    mode='lines+markers',
    marker=dict(color='#448CF2', size=5)
)


# Adding gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(tickformat=".1%")

# save
fig.write_image(os.path.join(output_dir, 'percent_all_homes_owned_by_investors.png'), width=IMG_WIDTH, height=IMG_HEIGHT)

# Show the plot
fig.show()

We can see that 1 in 5 homes in the San Diego market are owned by investors. That concentration is not evenly distributed among property types. Lets now analyze the ownership structure of Single Family Homes by portfolio size of the investors that own these properties. 

In [ ]:
# lets retrieve the portfolio size ownership metrics
portfolio_size_ownership = client.portfolio_metrics_sf_housing_stock_ownership.retrieve(
    parcl_id=analysis_parcl_id,
    as_dataframe=True
)

portfolio_size_ownership.head()

In [ ]:
# lets focus on the percentages, so we create a subset of the data
pso_pcts = portfolio_size_ownership[[
    'date', 
    'pct_sf_housing_stock_portfolio_2_to_9', 
    'pct_sf_housing_stock_portfolio_10_to_99', 
    'pct_sf_housing_stock_portfolio_100_to_999',
    'pct_sf_housing_stock_portfolio_1000_plus',
    'pct_sf_housing_stock_all_portfolios'
]]

# do some data wrangling to make the data long format and divide by 100 so plotly can handle the formatting
pso_pcts_long = pso_pcts.melt(id_vars=['date'], var_name='category', value_name='percentage')
pso_pcts_long['percentage'] = pso_pcts_long['percentage']/100

# rename to friendly labels
friendly_labels = {
    'pct_sf_housing_stock_portfolio_2_to_9': 'Investor Portfolio (2 to 9 Units)', 
    'pct_sf_housing_stock_portfolio_10_to_99': 'Investor Portfolio (10 to 99 Units)', 
    'pct_sf_housing_stock_portfolio_100_to_999': 'Investor Portfolio (100 to 999 Units)',
    'pct_sf_housing_stock_portfolio_1000_plus': 'Investor Portfolio (1000+ Units)',
    'pct_sf_housing_stock_all_portfolios': 'Investor Portfolio (All Sizes)'
}

pso_pcts_long['category'] = pso_pcts_long['category'].replace(friendly_labels)
pso_pcts_long.head()

In [ ]:
# Creating the bar chart
fig = px.bar(
    pso_pcts_long,
    x='category',
    y='percentage',
    title=f'Percentages of Single Family Housing Stock by Portfolio Size: {name}',
    labels={'percentage': '% of Single Family Homes Owned', 'category': 'Portfolio Size'},
    color_discrete_sequence=['#448CF2']  # Sets the color of the bars
)

# Adding layout image
fig.add_layout_image(
    labs_logo_dict
)

fig.update_layout(
    margin=dict(b=100),
    height=IMG_HEIGHT,
    width=IMG_WIDTH,
    legend=dict(
        x=0.01,
        y=0.99,
        traceorder="normal",
        xanchor='left',
        yanchor='top',
        title='Rolling Window (Days)' 
    ),
    title=PLOT_TITLE_SETTINGS,
    plot_bgcolor='#080D16', 
    paper_bgcolor='#080D16',
    font=dict(color='#FFFFFF'),
)

# Removing x-axis title as requested in previous query
fig.update_layout(xaxis_title="")
fig.update_yaxes(tickformat=".1%") 

# Adding gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# save image
fig.write_image(os.path.join(output_dir, 'portfolio_sfh_size_breakout.png'), width=IMG_WIDTH, height=IMG_HEIGHT)

# Show the plot
fig.show()

<a id='listings'></a>
#### New Listings Analysis
We can also capture the change in new listings on a 7, 30, 60 and 90 day rolling basis

In [ ]:
# first order of business, lets see how new listings have changed over the last year
start_date = '2023-01-01'

new_listings = client.for_sale_market_metrics_new_listings_rolling_counts.retrieve(
    parcl_id=analysis_parcl_id,
    start_date=start_date,
    as_dataframe=True,
    params={
        'limit': 300
    }
)
new_listings.head()

In [ ]:
# we will melt the data to make it long format
new_listings_long = pd.melt(
    new_listings, 
    id_vars=['date', 'parcl_id'], 
    value_vars=['rolling_30_day', 'rolling_60_day', 'rolling_90_day'],
    value_name='Number of New Listings',
    var_name='Rolling Window (Days)'
)

In [ ]:
# Creating the line chart where we show new listings for different rolling windows
fig = px.line(
    new_listings_long,
    x='date',
    y='Number of New Listings',
    color='Rolling Window (Days)',
    title=f'New Listings (Rolling Day Counts): {name}',
    labels={'count': 'Rolling Day Count'}
)

fig.add_layout_image(
    labs_logo_dict
)

fig.update_layout(
    margin=dict(b=100),
    width=IMG_WIDTH,
    height=IMG_HEIGHT,
    legend=dict(
        x=0.01,
        y=0.99,
        traceorder="normal",
        xanchor='left',
        yanchor='top',
        title='Rolling Window (Days)'
    ),
    title=PLOT_TITLE_SETTINGS,
    plot_bgcolor='#080D16',
    paper_bgcolor='#080D16',
    font=dict(color='#FFFFFF'),
    xaxis_title='date'
)

color_map = {
    'rolling_30_day': '#448CF2',
    'rolling_90_day': '#FFFFFF',
}

fig.update_traces(
    line=dict(width=3),
    mode='lines+markers',
    marker=dict(size=5)
)

for trace in fig.data:
    if trace.name in color_map:
        trace.line.color = color_map[trace.name]
        trace.marker.color = color_map[trace.name]

# Adding gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# save figure
fig.write_image(os.path.join(output_dir, 'new_listings_rolling_n_periods.png'), width=IMG_WIDTH, height=IMG_HEIGHT)


# Show the plot
fig.show()

We can also determine what percent of the new listings are coming from investors that are offloading properties.

In [ ]:
# of listings on the market, what percent of them are coming from investors?
investor_listings = client.investor_metrics_new_listings_for_sale_rolling_counts.retrieve(
    parcl_id=analysis_parcl_id,
    params={
        'limit': 300
    },
    as_dataframe=True
)
investor_listings.head(4)

In [ ]:
# we divide by 100 so plotly can handle the formatting, we also fiter out the rolling_7_day period
investor_listings['pct_for_sale_market'] = investor_listings['pct_for_sale_market']/100
investor_listings = investor_listings.loc[investor_listings['period'] != 'rolling_7_day']

In [ ]:
# Create the line chart
fig = px.line(
    investor_listings,
    x='date',
    y='pct_for_sale_market',
    color='period',  # This creates separate lines for each period
    title=f'Investor Share of New Resale Listings Activity: {name}',
    labels={'pct_for_sale_market': '% of New Listings by Investors'},
    line_shape='linear'  # 'spline' for smooth lines, if preferred
)

fig.add_layout_image(labs_logo_dict)

fig.update_layout(
    margin=dict(b=100),
    height=IMG_HEIGHT,
    width=IMG_WIDTH,
    legend=dict(
        x=0.01,
        y=0.99,
        traceorder="normal",
        xanchor='left',
        yanchor='top',
        title='Period (Days)'
    ),
    title=PLOT_TITLE_SETTINGS,
    plot_bgcolor='#080D16',
    paper_bgcolor='#080D16',
    font=dict(color='#FFFFFF'),
    xaxis_title='date' 
)

fig.update_yaxes(tickformat=".0%")

color_map = {
    'rolling_30_day': '#448CF2',
    'rolling_90_day': '#FFFFFF',
}

fig.update_traces(
    line=dict(width=3),
    mode='lines+markers',
    marker=dict(size=5)
)


for trace in fig.data:
    if trace.name in color_map:
        trace.line.color = color_map[trace.name]
        trace.marker.color = color_map[trace.name]

# Adding gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# save figure
fig.write_image(os.path.join(output_dir, 'investor_share_of_resale_listings_rolling_n_period.png'), width=IMG_WIDTH, height=IMG_HEIGHT)

# Show the plot
fig.show()


We can also filter out new listings by investors based on their portfolio size.

In [ ]:
# now lets get the listings by portfolio sizes in San Diego city as of this week
portfolio_metrics_new_listings_2_to_9 = client.portfolio_metrics_new_listings_for_sale_rolling_counts.retrieve(
        parcl_id=analysis_parcl_id,
        as_dataframe=True,
        portfolio_size='PORTFOLIO_2_TO_9',
    )

portfolio_metrics_new_listings_2_to_9.head()

In [ ]:
# repeat for portfolio size of 10 to 99
portfolio_metrics_new_listings_10_to_9 = client.portfolio_metrics_new_listings_for_sale_rolling_counts.retrieve(
        parcl_id=analysis_parcl_id,
        as_dataframe=True,
        portfolio_size='PORTFOLIO_10_TO_99',
    )

portfolio_metrics_new_listings_10_to_9.head()

In [ ]:
# repeat for portfolio size of 1000+
portfolio_metrics_new_listings_1000_plus = client.portfolio_metrics_new_listings_for_sale_rolling_counts.retrieve(
        parcl_id=analysis_parcl_id,
        as_dataframe=True,
        portfolio_size='PORTFOLIO_1000_PLUS',
    )

portfolio_metrics_new_listings_1000_plus.head()

<a id='prices'></a>
### Price Activity

Let's review rental prices, new listing prices and home sales prices. 

In [ ]:
# Prices
prices = client.market_metrics_housing_event_prices.retrieve(
    parcl_id=analysis_parcl_id,
    as_dataframe=True,
    params={
        'limit': 300
    }
)

# subset the data and melt it to long format
prices_long = prices[['date', 'parcl_id', 'price_median_sales', 'price_median_new_listings_for_sale']]

prices_long = pd.melt(
    prices_long, 
    id_vars=['date', 'parcl_id'], 
    value_vars=['price_median_sales', 'price_median_new_listings_for_sale'],
    value_name='Median Price ($)',
    var_name='Transaction Type'
)

# replace the variable names to make them more user friendly
r = {
    'price_median_sales': 'Median Sales Price',
    'price_median_new_listings_for_sale': 'Median New Listing Price'
}

prices_long['Transaction Type'] = prices_long['Transaction Type'].replace(r)

In [ ]:
# Creating the line chart
fig = px.line(
    prices_long,
    x='date',
    y='Median Price ($)',
    color='Transaction Type',
    title=f'Purchase Price vs. New Listings Median Prices ($): {name}',
)

fig.add_layout_image(
    labs_logo_dict
)

fig.update_layout(
    margin=dict(b=100),
    height=IMG_HEIGHT,
    width=IMG_WIDTH,
    title=PLOT_TITLE_SETTINGS,
    xaxis_title="Date",
    yaxis_title="Price ($)",
    yaxis_tickformat='$,', 
    xaxis_tickformat='%b %Y',
    plot_bgcolor='#080D16', 
    paper_bgcolor='#080D16',
    font=dict(color='#FFFFFF'),
    showlegend=True, 
    legend=dict(
        x=0.01,
        y=0.99,
        traceorder="normal",
        xanchor='left',
        yanchor='top'
    ),
)

color_map = {
    'Median Sales Price': '#448CF2',
    'Median New Listing Price': '#FFFFFF',
}

fig.update_traces(
    line=dict(width=3),
    mode='lines+markers',
    marker=dict(size=5)
)

for trace in fig.data:
    if trace.name in color_map:
        trace.line.color = color_map[trace.name]
        trace.marker.color = color_map[trace.name]

# Adding gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# save figure
fig.write_image(os.path.join(output_dir, 'purchase_price_vs_new_listings_price.png'), width=IMG_WIDTH, height=IMG_HEIGHT)

# Show the plot
fig.show()

In [ ]:
# we now do it for rental prices
fig = px.line(
    prices, 
    x='date', 
    y='price_median_new_rental_listings', 
    title=f'Median Rental Listing Price in {name}',  # Add city or location in title if needed
    labels={'price_median_new_rental_listings': 'Asking Rental Price ($)'},
    line_shape='linear'  # 'spline' for smooth lines, if preferred
)

# Adding layout image
fig.add_layout_image(labs_logo_dict)

fig.update_layout(
    margin=dict(b=100),
    height=IMG_HEIGHT,
    width=IMG_WIDTH,
    title=PLOT_TITLE_SETTINGS,
    xaxis_title="Date",
    yaxis_title="Price ($)",
    yaxis_tickformat='$,',
    xaxis_tickformat='%b %Y',
    plot_bgcolor='#080D16',
    paper_bgcolor='#080D16',
    font=dict(color='#FFFFFF'),
    showlegend=True,
    legend=dict(
        x=0.01,
        y=0.99,
        traceorder="normal",
        xanchor='left',
        yanchor='top'
    ),
)

fig.update_traces(
    line=dict(color='#FFFFFF', width=3),
    mode='lines+markers',
    marker=dict(color='#448CF2', size=5)
)

# Adding gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# save figure
fig.write_image(os.path.join(output_dir, 'median_rental_listings_price.png'), width=IMG_WIDTH, height=IMG_HEIGHT)

# Show the plot
fig.show()
